In [ ]:
import numpy as np
import nmrtools as nt
import matplotlib.pyplot as plt
from scipy.signal.windows import exponential


## Simulate a compound in frequency domain

In [ ]:
# frecuency in ppm units
f = np.linspace(0, 10, 64000) 
# larmor frequency for each multiplet
f0 = [1.46, 3.75] 
# coupling constants for each multiplet, in ppm units/
# a doublet and a quartet
j = [[0.01], [0.01, 0.01, 0.01]]  
# lambda = (1/t2) is related with the peak FWHM as follows FWHM = 2 lambda
# in ppm units for a FWHM = 0.0025 ppm, t2 = 800
t2 = [800.0, 800.0]
abundance = [3, 1]
sp = nt.simulation.simulate_compound_frequency(f, f0, j, t2, abundance)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(f, sp, linewidth=1)
ax.set_xlim(1.4, 4.0)
ax.invert_xaxis()
ax.set_xlabel("$\delta$ [ppm]")
ax.set_ylabel("Intensity [au]")

## Simulate a compound in time domain

In [ ]:
SW = 1024  # spectral width in Hz (Sampling Frequency)
N = 2 ** 10 # 1K points
t = np.arange(N) / SW # time vector in s

In [ ]:
f0 = [200, 400]
j = [[10], [10, 10, 10]]
abundance = [3, 1]
t2 = [2, 2]
fid = nt.simulation.simulate_compound_time(t, f0, j, t2, abundance)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(t, fid.real, linewidth=1)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Intensity [au]")
ax.set_title("Real part of the FID")

In [ ]:
# applying a window function to the FID
lb = 2 # line broadening in Hz
tau = SW / lb # tau is 1 / lb
w = exponential(N, center=0, tau=tau, sym=False)

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(t, fid.real * w, linewidth=1, label="windowed FID")
ax.plot(t, w * fid.real.max(), linewidth=1, label="window")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Intensity [au]")
ax.set_title("Apodization of a FID")
ax.legend()

In [ ]:
# Transform to frequency domain
N_zf = 2 * N  # size of the data after zero filing
sp = np.fft.fft(fid, n=N_zf)
sp = np.fft.fftshift(sp)
dt = 1 / SW  # sampling period
f = np.fft.fftfreq(N_zf, d=dt)
f = np.fft.fftshift(f)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(f, sp.real, linewidth=1)
ax.set_xlabel("Frequency [Hz]")
ax.set_ylabel("Intensity [au]")
ax.set_title("Spectrum")

## Simulate a mixture of compounds

In [ ]:
# Create spectra for multiple compounds

# frecuency in ppm units
f = np.linspace(0, 5, 4096) 
# larmor frequency for compound
f0 = [[1.46, 3.75], [3.21, 3.51]]
# coupling constants for each multiplet, in ppm units/
# a doublet and a quartet
j = [[[0.01], [0.01, 0.01, 0.01]], [[], []]] # empty list are used for singlets  
t2 = [[800.0, 800.0], [800, 800]]
abundance = [[3, 1], [1, 1]]
sp = nt.simulation.make_mixture_frequency(f, f0, j, t2, abundance)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
for k, ksp in enumerate(sp):
    ax.plot(f, ksp, linewidth=1, label="Compound {}".format(k + 1))
ax.set_xlim(1.4, 4.0)
ax.invert_xaxis()
ax.set_xlabel("$\delta$ [ppm]")
ax.set_ylabel("Intensity [au]")
ax.legend()

In [ ]:
# simulate mixtures
n_samples = 200
n_compounds = sp.shape[0]
mu = [5, 7]  # mean for each compound
sigma = [1, 2] # std for each compound
coeff = np.random.normal(size=(n_samples, n_compounds), loc=mu, scale=sigma)

sp_mix = np.dot(coeff, sp)

In [ ]:
# plot five different mixtures
y_offset = 5.0
fig, ax = plt.subplots(figsize=(8, 4))
for i, k in enumerate(np.random.choice(n_samples, 5, replace=False)):
    ax.plot(f, sp_mix[k] + y_offset * i, linewidth=1, label="Mix row={}".format(k))
ax.set_xlim(1.4, 4.0)
ax.invert_xaxis()
ax.set_xlabel("$\delta$ [ppm]")
ax.set_ylabel("Intensity [au]")
ax.legend()

## STOCSY analysis

In [ ]:
# peak detection in the mean spectra
sp_mean = sp_mix.mean(axis=0)
noise = nt.peaks.estimate_noise(sp_mean)
baseline = nt.peaks.estimate_baseline(sp_mean, noise)
start, apex, end = nt.peaks.detect_peaks(sp_mean, noise, baseline)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(f, sp_mean, linewidth=1, label="mean spectrum")
ax.scatter(f[apex], sp_mean[apex], marker="x", label="peaks")
ax.set_xlabel("Frequency [Hz]")
ax.set_ylabel("Intensity [au]")
ax.set_title("Spectrum")
ax.set_xlim(1.4, 4.0)
ax.invert_xaxis()
ax.set_xlabel("$\delta$ [ppm]")
ax.set_ylabel("Intensity [au]")
ax.legend()

In [ ]:
# STOCSY traces
dp = apex[0]  # driver peak
trace_cov = nt.utils.covmatk(sp_mix, dp)
trace_corr = nt.utils.corrmatk(sp_mix, dp)

fig, ax = plt.subplots(figsize=(8, 4))
nt.plot.plot_colored_curve(f, trace_cov, trace_corr, ax=ax, cmap="jet")
ax.scatter(f[dp], trace_cov[dp], label="Driver Peak", marker="x")
ax.set_xlim(1, 4)
ax.set_ylim(0, 10)
ax.set_xlabel("$\delta$ [ppm]")
ax.set_ylabel("Cov [au]")
ax.set_title("STOCSY trace @f2={:.3f} ppm".format(f[dp]))
cbar_loc = [0.4, 0.8, 0.2, 0.025]
ax.invert_xaxis()
nt.plot.add_colorbar(fig, cbar_loc, 0, 1, cmap="jet", orientation="horizontal", label="Correlation")
ax.legend()

In [ ]:
# 2D STOCSY plot
stocsy_cov = nt.utils.covmat(sp_mix)
stocsy_corr = nt.utils.corrmat(sp_mix)

fig, ax = plt.subplots()
levels = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
nt.plot.plot_colored_contours(f, f, stocsy_cov, stocsy_corr, levels, ax=ax, cmap="jet")
ax.set_xlim(1, 4)
ax.set_ylim(1, 4)
ax.set_xlabel("$\delta$ [ppm]")
ax.set_ylabel("$\delta$ [ppm]")
ax.set_title("STOCSY spectrum")
cbar_loc = [0.4, 0.8, 0.2, 0.025]
ax.invert_xaxis()
ax.invert_yaxis()
nt.plot.add_colorbar(fig, cbar_loc, 0, 1, cmap="jet", orientation="horizontal", label="Correlation")

## Multiplet annotation

In [ ]:
def list_to_str(l):
    l = ["{:.1f}".format(x) for x in l]
    return ", ".join(l)

In [ ]:
j_sim = [7, 7, 7]
f = np.linspace(-30, 30, 1000)
t2 = 2
sp = nt.multiplet.simulate_frequency(f, j_sim, t2)

# multiplet frequency and height estimation
noise = nt.peaks.estimate_noise(sp)
baseline = nt.peaks.estimate_baseline(sp, noise)
_, apex, _ = nt.peaks.detect_peaks(sp, noise, baseline)
f_peaks = f[apex]
h_peaks = sp[apex]

# multiplet annotation
f_tol = 1
j_annotation = nt.multiplet.annotate(f_peaks, h_peaks, f_tol)[0]

In [ ]:
fig, ax = plt.subplots()
ax.plot(f, sp)
sim_text = "$J_{sim.} (Hz) $: " + list_to_str(j_sim)
ann_text = "$J_{ann.} (Hz) $: " + list_to_str(j_annotation)
ax.set_xlabel("Frequency [Hz]")
ax.set_ylabel("Intensity [au]")
ax.annotate(sim_text, (10, 1.2))
ax.annotate(ann_text, (10, 1.0))

In [ ]:
fig, ax = plt.subplots(ncols=3, nrows=3, sharex=True, sharey=True, figsize=(9, 9))
simulated_j_list = [
    [7, 7, 7],
    [5, 7, 9],
    [5, 5, 8, 13],
    [4, 7, 7],
    [3, 5, 5, 5, 12],
    [3, 5],
    [8, 8, 8, 8, 8],
    [9, 9, 11, 13, 13],
    [5]
]


for k, j_sim, in enumerate(simulated_j_list):
    row, col = np.divmod(k, 3)
    sp = nt.multiplet.simulate_frequency(f, j_sim, t2)
    sp = sp / sp.max()
    
    # multiplet frequency and height estimation using peak detection
    noise = nt.peaks.estimate_noise(sp)
    baseline = nt.peaks.estimate_baseline(sp, noise)
    _, apex, _ = nt.peaks.detect_peaks(sp, noise, baseline)
    f_peaks = f[apex]
    h_peaks = sp[apex]

    # multiplet annotation
    f_tol = 1.5
    j_annotation = nt.multiplet.annotate(f_peaks, h_peaks, f_tol, max_perturbation=2)
    j_annotation = j_annotation[0]
    
    
    # plot
    cax = ax[row, col]
    sim_text = "$J_{sim.} (Hz) $: " + list_to_str(j_sim)
    ann_text = "$J_{ann.} (Hz) $: " + list_to_str(j_annotation)
    cax.plot(f, sp)
    cax.annotate(sim_text, (-30, 2.0))
    cax.annotate(ann_text, (-30, 1.5))
    cax.set_ylim(0, 2.5)
    
    if row == 2:
        cax.set_xlabel("Frequency [Hz]")
        
    if col == 0:
        cax.set_ylabel("Intensity [au]")
fig.tight_layout()